# 教檢自問自答

In [9]:
%pip install groq

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import gradio as gr
from groq import Groq

# 設定 API 金鑰，請自行確認金鑰是否正確
client = Groq(api_key="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")

def examiner_question():
    """
    呼叫 groq API 生成主考官的甄選題目，
    內容為針對 108 課綱提出教師甄選問題。
    """
    messages = [
        {
            "role": "user",
            "content": "你是一個講繁體中文的臺灣國中的教師甄選主考官，你需要提出一個甄選題目來詢問面試者。範圍包刮以下其一 1. 學校行政 2. 課程設計 3. 教學法 4. 評量與評鑑 5. 班級經營 6. 家長溝通 7. 教師專業成長 8. 教育政策與法規。"
        }
    ]
    chat_completion = client.chat.completions.create(
        messages=messages,
        model="llama-3.3-70b-versatile",
    )
    question = chat_completion.choices[0].message.content
    return question

def angel_suggestion(user_answer):
    """
    呼叫 groq API 生成小天使的回應，
    根據使用者回答進行錯誤找出、提出建議及提供其他可能的回答。
    若使用者回答為空，則先取得主考官的題目，再根據該題目直接提供一個示範回答。
    """
    if not user_answer.strip():
        # 當使用者回答為空時，取得主考官題目作為參考
        question = examiner_question()
        prompt = f"你是一個講繁體中文的小天使，請根據以下問題直接提供一個回答：{question}"
    else:
        prompt = f"你是一個講繁體中文的小天使，負責找出使用者回答中的錯誤、提出建議並提供其他可能的回答。請根據以下使用者的回答給出建議：{user_answer}"
    messages = [
        {
            "role": "user",
            "content": prompt
        }
    ]
    chat_completion = client.chat.completions.create(
        messages=messages,
        model="llama-3.3-70b-versatile",
    )
    suggestion = chat_completion.choices[0].message.content
    return suggestion

def generate_exam_question():
    return examiner_question()

def generate_angel_feedback(user_answer):
    return angel_suggestion(user_answer)

with gr.Blocks() as demo:
    gr.Markdown("## 教師甄選對話系統")
    
    gr.Markdown("### 主考官 - 生成甄選題目")
    with gr.Row():
        generate_btn = gr.Button("生成甄選題目")
        # 設定較多的 lines 及 max_lines 參數，讓文字框支援垂直捲軸
        exam_question_output = gr.Textbox(label="甄選題目", lines=10, max_lines=20)
    generate_btn.click(generate_exam_question, outputs=exam_question_output)
    
    gr.Markdown("### 小天使 - 提供建議")
    with gr.Row():
        user_answer_input = gr.Textbox(label="請輸入您的回答", lines=10, max_lines=20)
        feedback_btn = gr.Button("獲取建議")
        feedback_output = gr.Textbox(label="小天使建議", lines=10, max_lines=20)
    feedback_btn.click(generate_angel_feedback, inputs=user_answer_input, outputs=feedback_output)

if __name__ == "__main__":
    demo.launch()


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
